In [1]:
from ax.service.ax_client import AxClient, ObjectiveProperties 
from ax.utils.measurement.synthetic_functions import hartmann6 # function to optimize
from ax.utils.notebook.plotting import init_notebook_plotting, render 

In [2]:
ax_client = AxClient()

# experiment requires a "search space" (parameters and parameter constraints) and an "optimization configuration" (objective and outcome constraints)
ax_client.create_experiment(
    name="Hartmann6 experiment",

    # search space
    parameters=[
        {
            "name": "x1",
            "type": "range",
            "bounds": [0.0, 1.0],
            "value_type": "float",  # Optional, defaults to inference from type of "bounds".
            "log_scale": False,  # Optional, defaults to False.
        },
        {
            "name": "x2",
            "type": "range",
            "bounds": [0.0, 1.0],
            "value_type": "float",
        },
        {
            "name": "x3",
            "type": "range",
            "bounds": [0.0, 1.0],
            "value_type": "float",
        },
        {
            "name": "x4",
            "type": "range",
            "bounds": [0.0, 1.0],
            "value_type": "float",
        },
        {
            "name": "x5",
            "type": "range",
            "bounds": [0.0, 1.0],
            "value_type": "float",
        },
        {
            "name": "x6",
            "type": "range",
            "bounds": [0.0, 1.0],
            "value_type": "float",
        },
    ],

    #optimization configuration
    objectives={
        "hartmann6" : ObjectiveProperties(minimize=True)
    },
    parameter_constraints=["x1 + x2 <= 2.0"],  # Optional.
    outcome_constraints=["l2norm <= 1.25"],  # Optional.
)

[INFO 08-20 00:24:40] ax.generation_strategy.dispatch_utils: Using Generators.BOTORCH_MODULAR since there is at least one ordered parameter and there are no unordered categorical parameters.
[INFO 08-20 00:24:40] ax.generation_strategy.dispatch_utils: Using Bayesian Optimization generation strategy: GenerationStrategy(name='Sobol+BoTorch', steps=[Sobol for 12 trials, BoTorch for subsequent trials]). Iterations after 12 will take longer to generate due to model-fitting.


In [3]:
import numpy as np

# dummy evaluation function, returns output of harmann6 and l2norm
# takes experiment parameters as input
def evaluate(parameterization):
    x = np.array([parameterization.get(f"x{i+1}") for i in range(6)])

    # return mean value and standard error of mean (uncertainity)
    return {"hartmann6" : (hartmann6(x), 0.0), "l2norm" : (np.sqrt((x**2).sum()), 0.0)}

In [4]:
# optimization loop
n = 25
for i in range(n):
    parameterization, trial_index = ax_client.get_next_trial() # query client for new trial
    ax_client.complete_trial(trial_index=trial_index, raw_data=evaluate(parameterization)) # submit evaluation of new trial back to the client

[INFO 08-20 00:24:40] ax.service.ax_client: Generated new trial 0 with parameters {'x1': 0.155389, 'x2': 0.240024, 'x3': 0.756256, 'x4': 0.059617, 'x5': 0.785997, 'x6': 0.792401} using model Sobol.
[INFO 08-20 00:24:40] ax.service.ax_client: Completed trial 0 with data: {'hartmann6': (-0.385802, 0.0), 'l2norm': (1.379466, 0.0)}.
[INFO 08-20 00:24:40] ax.service.ax_client: Generated new trial 1 with parameters {'x1': 0.626774, 'x2': 0.625707, 'x3': 0.205438, 'x4': 0.82324, 'x5': 0.342676, 'x6': 0.153771} using model Sobol.
[INFO 08-20 00:24:40] ax.service.ax_client: Completed trial 1 with data: {'hartmann6': (-0.359568, 0.0), 'l2norm': (1.282715, 0.0)}.
[INFO 08-20 00:24:40] ax.service.ax_client: Generated new trial 2 with parameters {'x1': 0.789609, 'x2': 0.274704, 'x3': 0.520386, 'x4': 0.391232, 'x5': 0.151348, 'x6': 0.362064} using model Sobol.
[INFO 08-20 00:24:40] ax.service.ax_client: Completed trial 2 with data: {'hartmann6': (-0.314972, 0.0), 'l2norm': (1.129958, 0.0)}.
[INFO 08

In [5]:
ax_client.get_max_parallelism() # (num of trials, max parallelism)
# example [(12, 12), (-1, 3)]: "the max parallelism is 12 for the first 12 trials and 3 for all subsequent trials"

[(12, 12), (-1, 3)]

In [6]:
ax_client.get_trials_data_frame()

,trial_index,arm_name,trial_status,generation_node,l2norm,hartmann6,x1,x2,x3,x4,x5,x6
0,0,0_0,COMPLETED,GenerationStep_0,1.379466,-0.385802,1.553888e-01,2.400243e-01,0.756256,0.059617,0.785997,0.792401
1,1,1_0,COMPLETED,GenerationStep_0,1.282715,-0.359568,6.267743e-01,6.257067e-01,0.205438,0.823240,0.342676,0.153771
2,2,2_0,COMPLETED,GenerationStep_0,1.129958,-0.314972,7.896095e-01,2.747037e-01,0.520386,0.391232,0.151348,0.362064
3,3,3_0,COMPLETED,GenerationStep_0,1.593761,-0.030741,3.029803e-01,8.556910e-01,0.438317,0.726000,0.719486,0.692252
4,4,4_0,COMPLETED,GenerationStep_0,1.380939,-0.161939,4.540908e-01,3.845097e-01,0.020358,0.545190,0.537089,0.983277
5,5,5_0,COMPLETED,GenerationStep_0,1.705797,-0.010795,9.513633e-01,9.968766e-01,0.938410,0.337608,0.093509,0.086904
6,6,6_0,COMPLETED,GenerationStep_0,1.246836,-0.034272,6.009997e-01,9.887658e-02,0.256167,0.878744,0.402207,0.428999
7,7,7_0,COMPLETED,GenerationStep_0,1.424151,-0.368634,1.187874e-01,5.159233e-01,0.705470,0.238430,0.970605,0.501307
8,8,8_0,COMPLETED,GenerationStep_0,0.904074,-2.639563,5.880085e-02,3.223445e-01,0.355839,0.259670,0.294152,0.655294
9,9,9_0,COMPLETED,GenerationStep_0,1.605194,-0.884656,5.341542e-01,8.119106e-01,0.680020,0.623128,0.835007,0.290132
